In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from nnAudio.Spectrogram import CQT1992v2
from torch.utils.data import DataLoader, Dataset
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
import matplotlib.pyplot as plt
import sys
from functools import partial
sys.path.insert(0, "../")

In [2]:
from src.model import CustomModel_v1, Andrewnet_v3_true
from src.dataset import TrainDataset
from src.transforms import stack_bandpass_transform, stack_bandpass_turkey_transform, stack_turkey

ModuleNotFoundError: No module named 'src.model'

In [4]:
class CFG:
    def __init__(self):
        self.qtransform_params = {
            'sr': 2048,
            'fmin': 30,
            'fmax': 400,
            'hop_length': 64,
            'bins_per_octave': 12,
            'filter_scale': 0.5,
        }

        self.bandpass_param = {
          'lf': 30,
          'hf': 400,
          'order': 8, 
          'sr': 2048,
        }
cfg = CFG()

class CFG:
    def __init__(self):
        self.qtransform_params = {
            'sr': 2048,
            'fmin': 20,
            'fmax': 1024,
            'hop_length': 32,
            'bins_per_octave': 8,
        }

        self.bandpass_param = {
          'lf': 30,
          'hf': 400,
          'order': 8, 
          'sr': 2048,
        }
cfg = CFG()

In [5]:
import torch
import torch.nn as nn
import timm

class CustomModel_v1(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.model = timm.create_model("efficientnet_b0", pretrained=pretrained, in_chans=3, num_classes=1)
        self.cqt = CQT1992v2(**cfg.qtransform_params)
        self.h, self.w = 256, 256

    def forward(self, x):
        bs, ch, sig_len = x.shape
        x = x.view(-1, sig_len)
        x = self.cqt(x).unsqueeze(1)
        x = nn.functional.interpolate(x, (self.h, self.w))
        _, _, h, w = x.shape
        x = x.view(bs, 3, h, w)
        output = self.model(x)
        return output

In [6]:
INPUT_PATH = Path("/home/trytolose/rinat/kaggle/grav_waves_detection/input")

df = pd.read_csv(INPUT_PATH / "training_labels.csv")

files = list((INPUT_PATH / "train").rglob("*.npy"))
FILE_PATH_DICT = {x.stem: str(x) for x in files}
df["path"] = df["id"].apply(lambda x: FILE_PATH_DICT[x])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=69)
df["fold"] = -1
for f, (train_ids, val_ids) in enumerate(skf.split(df.index, y=df["target"])):
    df.loc[val_ids, "fold"] = f

In [1]:
!ls -lt ../weights/

total 695368
-rw-rw-r-- 1 trytolose trytolose   2560697 Aug 28 18:11 wavenet_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolose  17082923 Aug 28 13:00 cos_bandpass_minmax_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_turkey_q_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_turkey_transform_q_fold_4_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_turkey_transform_q_fold_3_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_turkey_transform_q_fold_2_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_turkey_transform_q_fold_1_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_turkey_transform_q_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolose  16327345 Aug 28 10:39 stack_bandpass_turkey_transform_cwt_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolose  16697067 Aug 28 10:39 stack_bandpass_transform_q_fold_0_w.pt
-rw-rw-r-- 1 trytolose trytolos

In [14]:
# model = CustomModel_v1(cfg).cuda()
model = Andrewnet_v3_true().cuda()
transform_f = partial(stack_bandpass_transform, params=cfg.bandpass_param)
dfs = []
for f in range(1):
#     model.load_state_dict(torch.load(f"../weights/cos_bandpass_minmax_fold_{f}_w.pt"))
    model.load_state_dict(torch.load(f"../weights/wavenet_fold_{f}_w.pt"))
    df_fold = df[df["fold"] == f].reset_index(drop=True)
    val_ds = TrainDataset(
        df_fold,
        mode="val",
        transform=transform_f,
    )

    val_loader = DataLoader(val_ds, shuffle=False, num_workers=12, batch_size=128, pin_memory=False)

    val_pred = []
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(val_loader, ncols=50, leave=True):
            x = x.cuda().float()
            y = y.cuda().float().unsqueeze(1)
            pred = model(x)
            pred = pred.sigmoid().cpu().data.numpy()
            val_pred.append(pred)
            
    val_pred = np.concatenate(val_pred).reshape(-1,)
    df_fold['pred'] = val_pred
    dfs.append(df_fold)
    break

100%|███████████| 875/875 [01:50<00:00,  7.88it/s]


In [15]:
df_pred_wav = pd.concat(dfs, ignore_index=True)

In [17]:
df_pred_cnn['pred_wave'] = df_pred_wav['pred']

In [21]:
xx = df_pred_cnn[(df_pred_cnn['target']==1) & (df_pred_cnn['pred']<0.5)]

xx[xx['pred_wave']>0.5]

,id,target,path,fold,pred,pred_wave
17,0008983698,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.437847,0.888049
115,004349ef0d,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.497625,0.803272
153,005646617c,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.452706,0.754645
195,006f545b8c,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.396070,0.511481
258,009ed4a8dc,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.397621,0.635463
...,...,...,...,...,...,...
111731,ff5f48f96e,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.461193,0.606963
111777,ff77bfb6d3,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.362596,0.507778
111796,ff832ec5e4,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.383760,0.712144
111907,ffc921782e,1,/home/trytolose/rinat/kaggle/grav_waves_detect...,0,0.393046,0.541465


In [23]:
df_pred = df_pred.drop("path", axis=1)

In [10]:
df_pred.to_csv("OOF_pred_turkey_bandpass.csv", index=False)

In [24]:
f0 = df_pred[df_pred['fold']==0]

metrics.roc_auc_score(f0['target'], f0['pred'])

0.8671333603182839

In [3]:
model = Andrewnet_v3_true()

In [4]:
waves = torch.rand(1, 3, 4096)
model(waves)

tensor([[-0.3139]], grad_fn=<AddmmBackward>)